**导入必要的软件库**
* cv2 : opencv-python 开源的计算机视觉库 
* numpy : 数值计算拓展库，用于矩阵等计算


In [1]:
# import the necessary packages
import cv2
import time
import numpy as np 

**加载预训练模型与网络定义文件**



In [2]:
pre_trained_model = "res10_300x300_ssd_iter_140000.caffemodel"
prototxt = "deploy.prototxt.txt"
# 通过dnn模块加载预训练的模型
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(prototxt,pre_trained_model)
print("model loaded!")

[INFO] loading model...
model loaded!


**加载待检测的图片**

In [3]:
imagename = "2.jpg"
# 将加载待检测的图片，大小设置为（300，300），并转化为所需格式.
image = cv2.imread(imagename)
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
	(300, 300), (104.0, 177.0, 123.0))

**进行人脸检测**

In [4]:
print("[INFO] computing object detections...")
# pass the blob through the network and obtain the detections and predictions
net.setInput(blob)
detections = net.forward()

[INFO] computing object detections...


**检测结果输出:**
* 1. 人脸框
* 2. 人脸的置信度


In [5]:
# 设置最小置信度
min_confidence = 0.5 

In [6]:

# loop over the detections
for i in range(0, detections.shape[2]):
	#提取结果中的置信度
	confidence = detections[0, 0, i, 2]

	# 滤除置信度比最小置信度较小的人脸
	if confidence > min_confidence:
		# 计算出人脸位置
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		# 画出人脸框架与显示人脸置信度
		text = "{:.2f}%".format(confidence * 100)
		y = startY - 10 if startY - 10 > 10 else startY + 10
		cv2.rectangle(image, (startX, startY), (endX, endY),
			(0, 0, 255), 2)
		cv2.putText(image, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

# 显示图片
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# initialize the video stream and allow the cammera sensor to warmup
print("[INFO] starting video stream...")
cap = cv2.VideoCapture(0)
time.sleep(2.0)


[INFO] starting video stream...


**利用摄像头进行实时的人脸检测人**

1. 读取摄像头数据

2. 图片格式转换

3. 利用模型进行人脸检测

4. 检测结果实时呈现在画面中

In [10]:
# loop over the frames from the video stream
while True:
    # 读取摄像头设局
	ret, frame = cap.read()
	# 将图像数据进行大小与格式转化
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,
		(300, 300), (104.0, 177.0, 123.0))
 
	# 将图片数据传递给神经网络
	net.setInput(blob)
	detections = net.forward()

	# 循环检测
	for i in range(0, detections.shape[2]):
		# 提取检测结果置信度
		confidence = detections[0, 0, i, 2]

		# 滤除置信度较低的检测结果
		if confidence < min_confidence:
			continue

		# 计算检测框的位置
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
 
		# 绘制检测框与置信度
		text = "{:.2f}%".format(confidence * 100)
		y = startY - 10 if startY - 10 > 10 else startY + 10
		cv2.rectangle(frame, (startX, startY), (endX, endY),
			(0, 0, 255), 2)
		cv2.putText(frame, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

	# 显示结果
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
 
	# 按q键结束
	if key == ord("q"):
		break

# 释放资源
cap.release()
cv2.destroyAllWindows()